In [4]:
import time
from urllib.request import FancyURLopener  # This is library that helps us create the headless browser
from random import choice #This library helps pick a random item from a list
import bs4 as bs
import datetime
from datetime import timedelta, date
import pandas as pd
from selenium import webdriver
import sys
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import numpy as np

c:\Users\dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
!pip install splinter

In [2]:
!pip install selenium

  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.4.26-py3-none-any.whl.metadata (2.5 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.4 MB ? eta -:--:--
   ------ --------------------------------- 1.6/9.4 MB 5.6 MB/s eta 0:00:02
   ------------ --------------------------- 2.9/9.4 MB 6.7 MB/s eta 0:00:01
   ------------- -------------------------- 3.1/9.4 MB 4.9 MB/s eta 0:00:02
   ---------------- ----------------------- 3.9/9.4 MB 4.7 MB/s eta 0:00:02
   ---------------------- ----------------- 5.2/9.4 MB 4.5 MB/s eta 0:00:01
   ------------------------- -------------- 6.0/9.4 MB 4.4 MB/s eta 0:00:01
   -------------------------------- ------- 7.6/9.4 MB 4.9 MB/s eta 0:00:01
   -------------------------------------- - 8.9/9.4 MB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 9.4

In [58]:
##def init_browser():
##    executable_path = {'executable_path': 'chromedriver.exe'}
##    return Browser("chrome", **executable_path, headless=False)
from splinter import Browser
from selenium.webdriver.chrome.service import Service

def init_browser():
    path = r'C:\Users\dell\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe'  # <-- use raw string (r'') or double backslashes
    service = Service(executable_path=path)
    return Browser("chrome", service=service, headless=False)



In [59]:
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.208 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.6367.208 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4 Safari/605.1.15',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:125.0) Gecko/20100101 Firefox/125.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edg/124.0.2478.80 Chrome/124.0.6367.208 Safari/537.36'
]

class MyOpener(FancyURLopener, object):
    version = choice(user_agents)

# Get each article and their link
def getArticleMeta(infotable):
    for i in range(0,len(infotable)):
        CallDesc.append(infotable[i].getText().encode('utf-8'))
        CallLink.append(infotable[i]['href'])
        
def getDate(i):
    date = i.find("span", class_= "date")
    date = date.getText()
    try:
        date = pd.to_datetime(date,infer_datetime_format=True)
    except ValueError:
        if "Yesterday" in date:
            Date = datetime.date.today() - timedelta(1)
            date = date[10:]
            date = str(Date) + date
        else:
            date = date.split(',')[1]
            date = date + ' ' + str(2017)
        date = pd.to_datetime(date,infer_datetime_format=True)
    return date

#def getTicker(companyName):
#    df = pd.read_csv('company.txt', sep = '\t')
#    df = df[df['Country'] == 'USA']
#    dfD = df.set_index('Name')['Ticker'].to_dict()
#    del df
#    return dfD[process.extractOne(companyName, dfD.keys())[0]]

In [60]:
# This uses the search tool on Seeking Alpha to get pull specific company's transcript.
def getTranscriptBySearch(ticker):
    CallDesc = []
    CallLink = []
    browser = init_browser()
    try:
        ticker = ticker
        url = 'https://seekingalpha.com/search/transcripts?term='+ ticker
        browser.visit(url) # Start the browser and open 'url'
    except KeyError:
        print("That company doesn't exist in my masterlist")
        raise

    assert 'Seeking Alpha' in browser.title # Wait for the page to load
    html = driver.page_source # Get the html of the page
    driver.quit() # Close the browser

    soup = bs.BeautifulSoup(html, 'html.parser')

    infotable = soup.find_all("div", class_= "transcript_link")
    for i in infotable:
        date = getDate(i)
        if (date >= datetime.datetime(2016,9,1)) & (ticker in i.find('a').getText()):
            CallDesc.append(i.find('a').getText().encode('utf-8'))
            CallLink.append(i.find('a')['href'])
    CreateTextFiles(CallLink)

In [61]:
# get list of S&P 100 companies and tickers from wikipedia

import pandas as pd # library for data analysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents

# get the response in the form of html
wikiurl="https://en.wikipedia.org/wiki/S%26P_100"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)

200


In [62]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
sandp100=soup.find_all('table')
df=pd.read_html(str(sandp100))

#convert list to dataframe
df=pd.DataFrame(df[2])
print(df.head(25))
df.to_csv('sandp100.csv', index=False)

   Symbol                          Name                  Sector
0    AAPL                    Apple Inc.  Information Technology
1    ABBV                        AbbVie             Health Care
2     ABT           Abbott Laboratories             Health Care
3     ACN                     Accenture  Information Technology
4    ADBE                    Adobe Inc.  Information Technology
5     AIG  American International Group              Financials
6     AMD        Advanced Micro Devices  Information Technology
7    AMGN                         Amgen             Health Care
8     AMT                American Tower             Real Estate
9    AMZN                        Amazon  Consumer Discretionary
10   AVGO                      Broadcom  Information Technology
11    AXP              American Express              Financials
12     BA                        Boeing             Industrials
13    BAC               Bank of America              Financials
14     BK                    BNY Mellon 

C:\Users\dell\AppData\Local\Temp\ipykernel_20632\3696830568.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html(str(sandp100))


In [63]:
!pip install lxml


In [64]:
# This goes into the transcript link and extracts the text from each link
def CreateTextFiles(CallLink):
    for link in range(0,len(CallLink)):
        myopener = MyOpener()
        page=myopener.open('https://seekingalpha.com' + CallLink[link])

        html = page.read()
        soup = bs.BeautifulSoup(html, 'lxml')

        infotable = soup.find_all("p", class_= "p")
        try:
            Output_File = open(str(infotable[0].find("a")['title']) + 
                               str(infotable[2].getText()) + ".txt",'w')
        except IndexError as exc:
            continue
        except TypeError as typ:
            continue

        for i in range(0,len(infotable)):
            if(infotable[i].getText().encode('utf-8').startswith('Copyright policy:')):
                break
            if(len(infotable[i].getText()) == 0):
                time.sleep(600)
                continue
            Output_File.write(infotable[i].getText().encode('utf-8'))
            Output_File.write("\n")
        Output_File.close()

In [66]:
import re
import datetime
import pandas as pd
from bs4 import BeautifulSoup as bs
import time

url = 'https://seekingalpha.com/symbol/AAPL/transcripts'
browser = init_browser()
browser.visit(url)
time.sleep(10)
html = browser.html
soup = bs(html, "html.parser")
results = soup.find_all("article")

for result in results:
    title = result.find('h3').text.strip()
    transcript_url = result.find('a')['href']
    date_str = result.find('span').text.strip()
    
    # Check if date string contains a 4-digit year before converting
    if re.search(r'\b\d{4}\b', date_str):
        try:
            date = pd.to_datetime(date_str, infer_datetime_format=True)
            if date > datetime.datetime(2022, 1, 1):
                print(title)
                print(transcript_url)
                print(date)
        except Exception as e:
            print(f"Skipping invalid date '{date_str}': {e}")
    else:
        print(f"Skipping date without year: {date_str}")

    


Skipping date without year: Earnings Call Insights
Skipping date without year: Earnings Call Insights
Skipping date without year: Thu, May 01
Skipping date without year: Thu, Jan. 30
Apple Inc. (AAPL) Q4 2024 Earnings Call Transcript
/article/4731856-apple-inc-aapl-q4-2024-earnings-call-transcript#source=section%3Atranscripts_and_insights%7Csection_asset%3Afull_transcripts%7Cfirst_level_url%3Asymbol%7Cbutton%3AApple%20Inc.%20(AAPL)%20Q4%202024%20Earnings%20Call%20Transcript%7Clock_status%3ANo%7Cline%3A3
2024-10-31 00:00:00
Apple Inc. (AAPL) Q3 2024 Earnings Call Transcript
/article/4709458-apple-inc-aapl-q3-2024-earnings-call-transcript#source=section%3Atranscripts_and_insights%7Csection_asset%3Afull_transcripts%7Cfirst_level_url%3Asymbol%7Cbutton%3AApple%20Inc.%20(AAPL)%20Q3%202024%20Earnings%20Call%20Transcript%7Clock_status%3ANo%7Cline%3A4
2024-08-01 00:00:00
Apple Inc. (AAPL) Q2 2024 Earnings Call Transcript
/article/4688870-apple-inc-aapl-q2-2024-earnings-call-transcript#source=se

C:\Users\dell\AppData\Local\Temp\ipykernel_20632\1861248627.py:23: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  date = pd.to_datetime(date_str, infer_datetime_format=True)


In [67]:
def getTranscriptInfo(ticker, browser_):   
    url = 'https://seekingalpha.com/symbol/' + ticker + '/transcripts'
    browser = browser_
    browser.visit(url)
    time.sleep(10)
    html = browser.html
    soup = bs(html, "html.parser")
    results = soup.find_all("article")
    search_results = []
    for result in results:
        try:
            title = result.find('h3').text
            url = result.find('a')['href']
            date = result.find('span').text
            date = pd.to_datetime(date,infer_datetime_format=True)
            if (date > datetime.datetime(2022, 1, 1)):
                print(title)
                print(url)
                print(date)
                print('----------------------------------------')
                results_dict = {
                    "title": title,
                    "url": url,
                    "date": date,
                    "ticker": ticker
                }
                search_results.append(results_dict)
        except:
            continue
    return search_results

In [9]:
browser = init_browser()
getTranscriptInfo('AAPL', browser)

Apple Inc. (AAPL) CEO Tim Cook on Q4 2020 Results - Earnings Call Transcript
/article/4382943-apple-inc-aapl-ceo-tim-cook-on-q4-2020-results-earnings-call-transcript?source=content_type%3Areact%7Csection%3ATranscripts%7CsectionAsset%3ATranscripts%7Cfirst_level_url%3Asymbol%7Cbutton%3AAuthor%7Clock_status%3ANo%7Cline%3A1
2020-10-29 00:00:00
----------------------------------------
Apple Inc. (AAPL) CEO Tim Cook on Q3 2020 Results - Earnings Call Transcript
/article/4362707-apple-inc-aapl-ceo-tim-cook-on-q3-2020-results-earnings-call-transcript?source=content_type%3Areact%7Csection%3ATranscripts%7CsectionAsset%3ATranscripts%7Cfirst_level_url%3Asymbol%7Cbutton%3AAuthor%7Clock_status%3ANo%7Cline%3A2
2020-07-30 00:00:00
----------------------------------------
Apple Inc. (AAPL) CEO Tim Cook on Q2 2020 Results - Earnings Call Transcript
/article/4341792-apple-inc-aapl-ceo-tim-cook-on-q2-2020-results-earnings-call-transcript?source=content_type%3Areact%7Csection%3ATranscripts%7CsectionAsset%3

[{'title': 'Apple Inc. (AAPL) CEO Tim Cook on Q4 2020 Results - Earnings Call Transcript',
  'url': '/article/4382943-apple-inc-aapl-ceo-tim-cook-on-q4-2020-results-earnings-call-transcript?source=content_type%3Areact%7Csection%3ATranscripts%7CsectionAsset%3ATranscripts%7Cfirst_level_url%3Asymbol%7Cbutton%3AAuthor%7Clock_status%3ANo%7Cline%3A1',
  'date': Timestamp('2020-10-29 00:00:00'),
  'ticker': 'AAPL'},
 {'title': 'Apple Inc. (AAPL) CEO Tim Cook on Q3 2020 Results - Earnings Call Transcript',
  'url': '/article/4362707-apple-inc-aapl-ceo-tim-cook-on-q3-2020-results-earnings-call-transcript?source=content_type%3Areact%7Csection%3ATranscripts%7CsectionAsset%3ATranscripts%7Cfirst_level_url%3Asymbol%7Cbutton%3AAuthor%7Clock_status%3ANo%7Cline%3A2',
  'date': Timestamp('2020-07-30 00:00:00'),
  'ticker': 'AAPL'},
 {'title': 'Apple Inc. (AAPL) CEO Tim Cook on Q2 2020 Results - Earnings Call Transcript',
  'url': '/article/4341792-apple-inc-aapl-ceo-tim-cook-on-q2-2020-results-earnings-

In [68]:
all_results = []
df = pd.read_csv('sandp100.csv', usecols = ['Symbol'])
tickers = df['Symbol'].tolist()
browser = init_browser()
for ticker in tickers:
    counter = 0
    results = getTranscriptInfo(ticker, browser)
    all_results = all_results + results
    time.sleep(5)
    
    

C:\Users\dell\AppData\Local\Temp\ipykernel_20632\3533487191.py:15: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  date = pd.to_datetime(date,infer_datetime_format=True)


Apple Inc. (AAPL) Q4 2024 Earnings Call Transcript
/article/4731856-apple-inc-aapl-q4-2024-earnings-call-transcript#source=section%3Atranscripts_and_insights%7Csection_asset%3Afull_transcripts%7Cfirst_level_url%3Asymbol%7Cbutton%3AApple%20Inc.%20(AAPL)%20Q4%202024%20Earnings%20Call%20Transcript%7Clock_status%3ANo%7Cline%3A3
2024-10-31 00:00:00
----------------------------------------
Apple Inc. (AAPL) Q3 2024 Earnings Call Transcript
/article/4709458-apple-inc-aapl-q3-2024-earnings-call-transcript#source=section%3Atranscripts_and_insights%7Csection_asset%3Afull_transcripts%7Cfirst_level_url%3Asymbol%7Cbutton%3AApple%20Inc.%20(AAPL)%20Q3%202024%20Earnings%20Call%20Transcript%7Clock_status%3ANo%7Cline%3A4
2024-08-01 00:00:00
----------------------------------------
Apple Inc. (AAPL) Q2 2024 Earnings Call Transcript
/article/4688870-apple-inc-aapl-q2-2024-earnings-call-transcript#source=section%3Atranscripts_and_insights%7Csection_asset%3Afull_transcripts%7Cfirst_level_url%3Asymbol%7Cbutt

KeyboardInterrupt: 

In [69]:
df_all = pd.DataFrame(all_results)

In [70]:
df_all.to_csv('transcript_list.csv', index=False)

In [71]:
def getTextFromArticle(url, browser_):
    url_begin = 'https://seekingalpha.com/'
    url_end = url
    browser = browser_
    browser.visit(url_begin+url_end)
    time.sleep(10)
    html = browser.html
    soup = bs(html, "html.parser")
    texts = soup.find_all('p')
    total_doc = ''
    for text in texts:
        total_doc += text.text + '\n'
    return total_doc


In [72]:
texts = soup.find_all('p')
total_doc = ''
for text in texts:
    total_doc += text.text + '\n'
print(total_doc)
    

Bull or Bear, Seeking Alpha is thereMarkets change, but our community always has the latest news, in-depth analysis, and powerful stock ratings.



In [74]:
df_all = pd.read_csv("transcript_list.csv")
df_ECT = df_all[df_all['title'].str.contains("Earnings Call Transcript")]
df_ECT["text"] = np.nan
df_ECT

,title,url,date,ticker,text
0,Apple Inc. (AAPL) Q4 2024 Earnings Call Transc...,/article/4731856-apple-inc-aapl-q4-2024-earnin...,2024-10-31,AAPL,NaN
1,Apple Inc. (AAPL) Q3 2024 Earnings Call Transc...,/article/4709458-apple-inc-aapl-q3-2024-earnin...,2024-08-01,AAPL,NaN
2,Apple Inc. (AAPL) Q2 2024 Earnings Call Transc...,/article/4688870-apple-inc-aapl-q2-2024-earnin...,2024-05-02,AAPL,NaN
3,Apple Inc. (AAPL) Q1 2024 Earnings Call Transc...,/article/4666956-apple-inc-aapl-q1-2024-earnin...,2024-02-01,AAPL,NaN
4,Apple Inc. (AAPL) Q4 2023 Earnings Call Transc...,/article/4646927-apple-inc-aapl-q4-2023-earnin...,2023-11-02,AAPL,NaN
5,Apple Inc. (AAPL) Q3 2023 Earnings Call Transc...,/article/4623702-apple-inc-aapl-q3-2023-earnin...,2023-08-03,AAPL,NaN
6,Apple (AAPL) Q2 2023 Earnings Call Transcript,/article/4600254-apple-aapl-q2-2023-earnings-c...,2023-05-04,AAPL,NaN
7,"Apple, Inc. (AAPL) Q1 2023 Earnings Call Trans...",/article/4574934-apple-inc-aapl-q1-2023-earnin...,2023-02-02,AAPL,NaN
8,Apple Inc. (AAPL) Q4 2022 Earnings Call Transc...,/article/4550156-apple-inc-aapl-q4-2022-earnin...,2022-10-27,AAPL,NaN
9,"Apple, Inc. (AAPL) CEO Tim Cook on Q3 2022 Res...",/article/4527231-apple-inc-aapl-ceo-tim-cook-o...,2022-07-28,AAPL,NaN


In [75]:
df_ECT.iloc[0]['url']

'/article/4731856-apple-inc-aapl-q4-2024-earnings-call-transcript#source=section%3Atranscripts_and_insights%7Csection_asset%3Afull_transcripts%7Cfirst_level_url%3Asymbol%7Cbutton%3AApple%20Inc.%20(AAPL)%20Q4%202024%20Earnings%20Call%20Transcript%7Clock_status%3ANo%7Cline%3A3'

In [76]:
print('')

In [78]:
browser = init_browser()
text = getTextFromArticle(df_ECT.iloc[0]['url'],browser)




In [79]:
df_ECT.iloc[0]['text'] = getTextFromArticle(df_ECT.iloc[0]['url'],browser)

C:\Users\dell\AppData\Local\Temp\ipykernel_20632\204936955.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ECT.iloc[0]['text'] = getTextFromArticle(df_ECT.iloc[0]['url'],browser)


In [81]:
df_ECT.head(20)

,title,url,date,ticker,text
0,Apple Inc. (AAPL) Q4 2024 Earnings Call Transc...,/article/4731856-apple-inc-aapl-q4-2024-earnin...,2024-10-31,AAPL,NaN
1,Apple Inc. (AAPL) Q3 2024 Earnings Call Transc...,/article/4709458-apple-inc-aapl-q3-2024-earnin...,2024-08-01,AAPL,NaN
2,Apple Inc. (AAPL) Q2 2024 Earnings Call Transc...,/article/4688870-apple-inc-aapl-q2-2024-earnin...,2024-05-02,AAPL,NaN
3,Apple Inc. (AAPL) Q1 2024 Earnings Call Transc...,/article/4666956-apple-inc-aapl-q1-2024-earnin...,2024-02-01,AAPL,NaN
4,Apple Inc. (AAPL) Q4 2023 Earnings Call Transc...,/article/4646927-apple-inc-aapl-q4-2023-earnin...,2023-11-02,AAPL,NaN
5,Apple Inc. (AAPL) Q3 2023 Earnings Call Transc...,/article/4623702-apple-inc-aapl-q3-2023-earnin...,2023-08-03,AAPL,NaN
6,Apple (AAPL) Q2 2023 Earnings Call Transcript,/article/4600254-apple-aapl-q2-2023-earnings-c...,2023-05-04,AAPL,NaN
7,"Apple, Inc. (AAPL) Q1 2023 Earnings Call Trans...",/article/4574934-apple-inc-aapl-q1-2023-earnin...,2023-02-02,AAPL,NaN
8,Apple Inc. (AAPL) Q4 2022 Earnings Call Transc...,/article/4550156-apple-inc-aapl-q4-2022-earnin...,2022-10-27,AAPL,NaN
9,"Apple, Inc. (AAPL) CEO Tim Cook on Q3 2022 Res...",/article/4527231-apple-inc-aapl-ceo-tim-cook-o...,2022-07-28,AAPL,NaN


In [82]:
text = getTextFromArticle('/article/4338017-abbott-laboratories-abt-ceo-robert-ford-on-q1-2020-results-earnings-call-transcript', browser)

In [83]:
for index, row in df_ECT.iloc[150:].iterrows():
    try:
        text = getTextFromArticle(row['url'],browser)
        time.sleep(1)
        file = open(row['title'] + '.txt','w')
        time.sleep(1)
        file.writelines(text) 
        file.close()
    except:
        continue
    


In [84]:
df_ECT.iloc[150:]

,title,url,date,ticker,text


In [85]:
wikiurl="https://en.wikipedia.org/wiki/S%26P_100"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)
print(response.status_code)
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
sandp100=soup.find_all('table')
df=pd.read_html(str(sandp100))

#convert list to dataframe
df=pd.DataFrame(df[2])
print(df.head(25))
df.to_csv('sandp100.csv', index=False)

200
   Symbol                          Name                  Sector
0    AAPL                    Apple Inc.  Information Technology
1    ABBV                        AbbVie             Health Care
2     ABT           Abbott Laboratories             Health Care
3     ACN                     Accenture  Information Technology
4    ADBE                    Adobe Inc.  Information Technology
5     AIG  American International Group              Financials
6     AMD        Advanced Micro Devices  Information Technology
7    AMGN                         Amgen             Health Care
8     AMT                American Tower             Real Estate
9    AMZN                        Amazon  Consumer Discretionary
10   AVGO                      Broadcom  Information Technology
11    AXP              American Express              Financials
12     BA                        Boeing             Industrials
13    BAC               Bank of America              Financials
14     BK                    BNY Mel

C:\Users\dell\AppData\Local\Temp\ipykernel_20632\2632633462.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df=pd.read_html(str(sandp100))


In [86]:
import pandas as pd # library for data analysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents

wikiurl2="https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
response=requests.get(wikiurl2)
print(response.status_code)
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
sandp500=soup.find_all('table')
df2=pd.read_html(str(sandp500))

#convert list to dataframe
#df2=pd.DataFrame(df[2])
#print(df.head(25))
#df.to_csv('sandp100.csv', index=False)

200


C:\Users\dell\AppData\Local\Temp\ipykernel_20632\957275720.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df2=pd.read_html(str(sandp500))


In [87]:
sandp100_df = pd.read_csv('sandp100.csv')
sandp500_df = df2[0]
sandp100_df_merged = sandp100_df.merge(sandp500_df, how='left', left_on='Symbol', right_on='Symbol').drop(columns=['Security', 'SEC filings'])
sandp100_df_merged.to_csv('sandp100_more_detail.csv', index=False)

KeyError: "['SEC filings'] not found in axis"

In [88]:
import pandas as pd # library for data analysis
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents

wikiurl2="https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
response=requests.get(wikiurl2)
print(response.status_code)
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
sandp500=soup.find_all('table')
df2=pd.read_html(str(sandp500))

#convert list to dataframe
#df2=pd.DataFrame(df[2])
#print(df.head(25))
#df.to_csv('sandp100.csv', index=False)

200


C:\Users\dell\AppData\Local\Temp\ipykernel_20632\957275720.py:11: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df2=pd.read_html(str(sandp500))


In [89]:
df2[0].to_csv('sandp500.csv', index=False)

In [90]:
all_results = []
df = pd.read_csv('sandp500.csv', usecols = ['Symbol'])
tickers = df['Symbol'].tolist()
browser = init_browser()
for ticker in tickers:
    results = getTranscriptInfo(ticker, browser)
    all_results = all_results + results
    time.sleep(2)

KeyboardInterrupt: 

In [15]:
df_ECT500 = pd.DataFrame(all_results)

In [16]:
df_ECT500

,title,url,date,ticker
0,3M Company (MMM) CEO Mike Roman on Q3 2020 Res...,/article/4381810-3m-company-mmm-ceo-mike-roman...,2020-10-27,MMM
1,3M Company 2020 Q3 - Results - Earnings Call P...,/article/4381577-3m-company-2020-q3-results-ea...,2020-10-27,MMM
2,"3M (MMM) CEO, Michael Roman on Q2 2020 Results...",/article/4361384-3m-mmm-ceo-michael-roman-on-q...,2020-07-28,MMM
3,3M Company 2020 Q2 - Results - Earnings Call P...,/article/4361248-3m-company-2020-q2-results-ea...,2020-07-28,MMM
4,3M Company (MMM) CEO Michael Roman on UBS Glob...,/article/4351738-3m-company-mmm-ceo-michael-ro...,2020-06-03,MMM
...,...,...,...,...
9773,Zoetis (ZTS) Q2 2018 Results - Earnings Call T...,/article/4193916-zoetis-zts-q2-2018-results-ea...,2018-08-02,ZTS
9774,Zoetis 2018 Q2 - Results - Earnings Call Slides,/article/4193869-zoetis-2018-q2-results-earnin...,2018-08-02,ZTS
9775,Zoetis (ZTS) Investor Presentation - Slideshow,/article/4174742-zoetis-zts-investor-presentat...,2018-05-16,ZTS
9776,Zoetis (ZTS) Q1 2018 Results - Earnings Call T...,/article/4168618-zoetis-zts-q1-2018-results-ea...,2018-05-02,ZTS


In [18]:
df_ECT500.to_csv('transcript_list_500.csv', index=False)

In [19]:

df_ECT500 = df_ECT500[df_ECT500['title'].str.contains("Earnings Call Transcript")]
df_ECT500

,title,url,date,ticker
0,3M Company (MMM) CEO Mike Roman on Q3 2020 Res...,/article/4381810-3m-company-mmm-ceo-mike-roman...,2020-10-27,MMM
2,"3M (MMM) CEO, Michael Roman on Q2 2020 Results...",/article/4361384-3m-mmm-ceo-michael-roman-on-q...,2020-07-28,MMM
6,3M Company (MMM) CEO Mike Roman on Q1 2020 Res...,/article/4340658-3m-company-mmm-ceo-mike-roman...,2020-04-28,MMM
9,3M's (MMM) CEO Mike Roman on Q4 2019 Results -...,/article/4319588-3ms-mmm-ceo-mike-roman-on-q4-...,2020-01-28,MMM
12,3M Company (MMM) CEO Mike Roman on Q3 2019 Res...,/article/4298957-3m-company-mmm-ceo-mike-roman...,2019-10-24,MMM
...,...,...,...,...
9768,"Zoetis, Inc. (ZTS) CEO Juan Ramon Alaix on Q1 ...",/article/4259182-zoetis-inc-zts-ceo-juan-ramon...,2019-05-02,ZTS
9769,"Zoetis, Inc. (ZTS) CEO Juan Ramón Alaix on Q4 ...",/article/4241305-zoetis-inc-zts-ceo-juan-ramon...,2019-02-14,ZTS
9771,Zoetis (ZTS) Q3 2018 Results - Earnings Call T...,/article/4217256-zoetis-zts-q3-2018-results-ea...,2018-11-01,ZTS
9773,Zoetis (ZTS) Q2 2018 Results - Earnings Call T...,/article/4193916-zoetis-zts-q2-2018-results-ea...,2018-08-02,ZTS


In [23]:
sandp500_df

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
502,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [24]:
df_ECT500

,title,url,date,ticker
0,3M Company (MMM) CEO Mike Roman on Q3 2020 Res...,/article/4381810-3m-company-mmm-ceo-mike-roman...,2020-10-27,MMM
2,"3M (MMM) CEO, Michael Roman on Q2 2020 Results...",/article/4361384-3m-mmm-ceo-michael-roman-on-q...,2020-07-28,MMM
6,3M Company (MMM) CEO Mike Roman on Q1 2020 Res...,/article/4340658-3m-company-mmm-ceo-mike-roman...,2020-04-28,MMM
9,3M's (MMM) CEO Mike Roman on Q4 2019 Results -...,/article/4319588-3ms-mmm-ceo-mike-roman-on-q4-...,2020-01-28,MMM
12,3M Company (MMM) CEO Mike Roman on Q3 2019 Res...,/article/4298957-3m-company-mmm-ceo-mike-roman...,2019-10-24,MMM
...,...,...,...,...
9768,"Zoetis, Inc. (ZTS) CEO Juan Ramon Alaix on Q1 ...",/article/4259182-zoetis-inc-zts-ceo-juan-ramon...,2019-05-02,ZTS
9769,"Zoetis, Inc. (ZTS) CEO Juan Ramón Alaix on Q4 ...",/article/4241305-zoetis-inc-zts-ceo-juan-ramon...,2019-02-14,ZTS
9771,Zoetis (ZTS) Q3 2018 Results - Earnings Call T...,/article/4217256-zoetis-zts-q3-2018-results-ea...,2018-11-01,ZTS
9773,Zoetis (ZTS) Q2 2018 Results - Earnings Call T...,/article/4193916-zoetis-zts-q2-2018-results-ea...,2018-08-02,ZTS


In [25]:
sandp500_df_merged = df_ECT500.merge(sandp500_df, how='left', left_on='ticker', right_on='Symbol').drop(columns=['Security', 'SEC filings'])

In [27]:
df_ECT500 = sandp500_df_merged.drop(columns=['ticker', 'CIK', 'Founded'])
df_ECT500

,title,url,date,Symbol,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added
0,3M Company (MMM) CEO Mike Roman on Q3 2020 Res...,/article/4381810-3m-company-mmm-ceo-mike-roman...,2020-10-27,MMM,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09
1,"3M (MMM) CEO, Michael Roman on Q2 2020 Results...",/article/4361384-3m-mmm-ceo-michael-roman-on-q...,2020-07-28,MMM,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09
2,3M Company (MMM) CEO Mike Roman on Q1 2020 Res...,/article/4340658-3m-company-mmm-ceo-mike-roman...,2020-04-28,MMM,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09
3,3M's (MMM) CEO Mike Roman on Q4 2019 Results -...,/article/4319588-3ms-mmm-ceo-mike-roman-on-q4-...,2020-01-28,MMM,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09
4,3M Company (MMM) CEO Mike Roman on Q3 2019 Res...,/article/4298957-3m-company-mmm-ceo-mike-roman...,2019-10-24,MMM,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09
...,...,...,...,...,...,...,...,...
5268,"Zoetis, Inc. (ZTS) CEO Juan Ramon Alaix on Q1 ...",/article/4259182-zoetis-inc-zts-ceo-juan-ramon...,2019-05-02,ZTS,Health Care,Pharmaceuticals,"Florham Park, New Jersey",2013-06-21
5269,"Zoetis, Inc. (ZTS) CEO Juan Ramón Alaix on Q4 ...",/article/4241305-zoetis-inc-zts-ceo-juan-ramon...,2019-02-14,ZTS,Health Care,Pharmaceuticals,"Florham Park, New Jersey",2013-06-21
5270,Zoetis (ZTS) Q3 2018 Results - Earnings Call T...,/article/4217256-zoetis-zts-q3-2018-results-ea...,2018-11-01,ZTS,Health Care,Pharmaceuticals,"Florham Park, New Jersey",2013-06-21
5271,Zoetis (ZTS) Q2 2018 Results - Earnings Call T...,/article/4193916-zoetis-zts-q2-2018-results-ea...,2018-08-02,ZTS,Health Care,Pharmaceuticals,"Florham Park, New Jersey",2013-06-21


In [28]:
df_ECT500 = df_ECT500.drop(columns=['Headquarters Location', 'Date first added'])
df_ECT500

,title,url,date,Symbol,GICS Sector,GICS Sub-Industry
0,3M Company (MMM) CEO Mike Roman on Q3 2020 Res...,/article/4381810-3m-company-mmm-ceo-mike-roman...,2020-10-27,MMM,Industrials,Industrial Conglomerates
1,"3M (MMM) CEO, Michael Roman on Q2 2020 Results...",/article/4361384-3m-mmm-ceo-michael-roman-on-q...,2020-07-28,MMM,Industrials,Industrial Conglomerates
2,3M Company (MMM) CEO Mike Roman on Q1 2020 Res...,/article/4340658-3m-company-mmm-ceo-mike-roman...,2020-04-28,MMM,Industrials,Industrial Conglomerates
3,3M's (MMM) CEO Mike Roman on Q4 2019 Results -...,/article/4319588-3ms-mmm-ceo-mike-roman-on-q4-...,2020-01-28,MMM,Industrials,Industrial Conglomerates
4,3M Company (MMM) CEO Mike Roman on Q3 2019 Res...,/article/4298957-3m-company-mmm-ceo-mike-roman...,2019-10-24,MMM,Industrials,Industrial Conglomerates
...,...,...,...,...,...,...
5268,"Zoetis, Inc. (ZTS) CEO Juan Ramon Alaix on Q1 ...",/article/4259182-zoetis-inc-zts-ceo-juan-ramon...,2019-05-02,ZTS,Health Care,Pharmaceuticals
5269,"Zoetis, Inc. (ZTS) CEO Juan Ramón Alaix on Q4 ...",/article/4241305-zoetis-inc-zts-ceo-juan-ramon...,2019-02-14,ZTS,Health Care,Pharmaceuticals
5270,Zoetis (ZTS) Q3 2018 Results - Earnings Call T...,/article/4217256-zoetis-zts-q3-2018-results-ea...,2018-11-01,ZTS,Health Care,Pharmaceuticals
5271,Zoetis (ZTS) Q2 2018 Results - Earnings Call T...,/article/4193916-zoetis-zts-q2-2018-results-ea...,2018-08-02,ZTS,Health Care,Pharmaceuticals


In [29]:
df_ECT500.to_csv('transcript_list_500.csv', index=False)

In [33]:
for index, row in df_ECT500.iterrows():
    try:
        text = getTextFromArticle(row['url'],browser)
        time.sleep(1)
        file = open(row['title'] + '.txt','w')
        time.sleep(1)
        file.writelines(text) 
        file.close()
    except:
        continue

In [31]:
df_ECT500

,title,url,date,Symbol,GICS Sector,GICS Sub-Industry
0,3M Company (MMM) CEO Mike Roman on Q3 2020 Res...,/article/4381810-3m-company-mmm-ceo-mike-roman...,2020-10-27,MMM,Industrials,Industrial Conglomerates
1,"3M (MMM) CEO, Michael Roman on Q2 2020 Results...",/article/4361384-3m-mmm-ceo-michael-roman-on-q...,2020-07-28,MMM,Industrials,Industrial Conglomerates
2,3M Company (MMM) CEO Mike Roman on Q1 2020 Res...,/article/4340658-3m-company-mmm-ceo-mike-roman...,2020-04-28,MMM,Industrials,Industrial Conglomerates
3,3M's (MMM) CEO Mike Roman on Q4 2019 Results -...,/article/4319588-3ms-mmm-ceo-mike-roman-on-q4-...,2020-01-28,MMM,Industrials,Industrial Conglomerates
4,3M Company (MMM) CEO Mike Roman on Q3 2019 Res...,/article/4298957-3m-company-mmm-ceo-mike-roman...,2019-10-24,MMM,Industrials,Industrial Conglomerates
...,...,...,...,...,...,...
5268,"Zoetis, Inc. (ZTS) CEO Juan Ramon Alaix on Q1 ...",/article/4259182-zoetis-inc-zts-ceo-juan-ramon...,2019-05-02,ZTS,Health Care,Pharmaceuticals
5269,"Zoetis, Inc. (ZTS) CEO Juan Ramón Alaix on Q4 ...",/article/4241305-zoetis-inc-zts-ceo-juan-ramon...,2019-02-14,ZTS,Health Care,Pharmaceuticals
5270,Zoetis (ZTS) Q3 2018 Results - Earnings Call T...,/article/4217256-zoetis-zts-q3-2018-results-ea...,2018-11-01,ZTS,Health Care,Pharmaceuticals
5271,Zoetis (ZTS) Q2 2018 Results - Earnings Call T...,/article/4193916-zoetis-zts-q2-2018-results-ea...,2018-08-02,ZTS,Health Care,Pharmaceuticals
